<a href="https://colab.research.google.com/github/alexzhab/image-processing/blob/main/detectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt 
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

import keras

from keras.utils import np_utils
from keras.optimizers import Adam

import os
import numpy as np
import itertools

import cv2
import os
%matplotlib inline

In [ ]:
import cv2

DETECTOR_METHOD_FAST = "fast"
DETECTOR_METHOD_SIFT = "sift"
DETECTOR_METHOD_HARRIS = "harris"

DESCRIPTOR_METHOD_SIFT = "sift"

def get_image_description(img, filename, threshold, detector, 
                          descriptor = DESCRIPTOR_METHOD_SIFT,
                          drawDebug = False):
    img_array = img_to_array(img)
    img_grayscale = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY).astype(np.uint8)
    
    if detector == DETECTOR_METHOD_HARRIS:
        keypoints = []
        responses = cv2.cornerHarris(img_grayscale, 2, 3, 0.04)
        for x in (-responses.flatten()).argsort()[:threshold]:
            indices = np.unravel_index(x, responses.shape)
            keypoints.append(cv2.KeyPoint(indices[1].astype(float), indices[0].astype(float), 17))
    elif detector == DETECTOR_METHOD_SIFT:
        feat = cv2.xfeatures2d.SIFT_create(threshold)
        keypoints = feat.detect(img_grayscale, None)
    elif detector == DETECTOR_METHOD_FAST:
        feat = cv2.FastFeatureDetector_create(threshold)
        keypoints = feat.detect(img_grayscale, None)
    else:
        raise Exception("Unknown detector")

    if drawDebug:
        img_array_kp = img_array.copy().astype(np.uint8)
        cv2.drawKeypoints(img_array_kp, keypoints, img_array_kp, (255, 0, 0))
        plt.axis('off')
        plt.imshow(img_array_kp)
        plt.savefig(filename)
        plt.close()
    
    descriptor = cv2.xfeatures2d.SIFT_create()
    return img_array, descriptor.compute(img_grayscale, keypoints)

In [ ]:
def find_filepaths(data_path: str) -> list:
    res_data = []
    for filename in os.listdir(data_path):
        if filename.endswith("jpg"):
            filepath = os.path.join(data_path, filename)
            res_data.append(filepath)
    return res_data


pics = find_filepaths("/content/pics/")
print(pics)
for i in range(len(pics)):
  img = cv2.imread(pics[i])
  path = pics[i].replace("pics", "harris")
  print(path)
  tmp, tmp_desc = get_image_description(load_img(pics[i]), path, threshold = 200, detector = DETECTOR_METHOD_HARRIS, drawDebug = True)